In [1]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [2]:
pip install bert-tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Downloading tokenization script created by the Google
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, Conv1D, Dropout, Input, LSTM, concatenate
from tensorflow.keras.models import Model
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

from bert import tokenization

In [5]:
img_feature = np.load('Emb_feature/new_test_features/.npy')
np.shape(img_feature)

(8313, 100, 1024)

In [7]:
FMS = pd.read_csv("FMS_final.csv")
FMS['textNdesc'] = 'In the picture '+ FMS.gen_caption + ' And the text says: ' + FMS.text
print(FMS.textNdesc[0], '\n\n')
print(len(FMS))


In the picture man in black and white cap is holding up sign . And the text says: its their character not their color that matters 


8313


In [8]:
FMS

,id,img,label,text,image,gen_caption,textNdesc
0,42953,img/42953.png,0,its their character not their color that matters,42953.png,man in black and white cap is holding up sign .,In the picture man in black and white cap is h...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058.png,man wearing black shirt and white shirt is sta...,In the picture man wearing black shirt and whi...
2,13894,img/13894.png,0,putting bows on your pet,13894.png,small child is jumping over the snow .,In the picture small child is jumping over the...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408.png,black dog with blue collar is walking through ...,In the picture black dog with blue collar is w...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403.png,two men are standing in front of white building .,In the picture two men are standing in front o...
...,...,...,...,...,...,...,...
8308,10423,img/10423.png,1,nobody wants to hang auschwitz me,10423.png,two men are sitting on the ground and one is w...,In the picture two men are sitting on the grou...
8309,98203,img/98203.png,1,when god grants you a child after 20 years of ...,98203.png,man with sunglasses and sunglasses is holding ...,In the picture man with sunglasses and sunglas...
8310,36947,img/36947.png,1,gays on social media: equality! body positivit...,36947.png,woman with long hair and black hair is smiling .,In the picture woman with long hair and black ...
8311,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,16492.png,two men wearing black and white shirts and whi...,In the picture two men wearing black and white...


In [9]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [10]:
# def build_model(bert_layer, max_len=512):
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
#     segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

#     image_input = tf.keras.Input(shape=(100,1024), batch_size=None, name="image_input")
#     image_flatten = tf.keras.layers.Flatten()(image_input)
#     image_dense_1 = tf.keras.layers.Dense(4096, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=54))(image_flatten)
#     image_dense_2 = tf.keras.layers.Dense(1024, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=32))(image_dense_1)


#     _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

#     clf_output = sequence_output[:, 0, :]

#     print(np.shape(sequence_output))
#     print(np.shape(clf_output))

#     image_question = tf.keras.layers.Multiply()([image_dense_2, clf_output])
#     image_question_dense_1 = tf.keras.layers.Dense(1000, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=19))(image_question)
#     image_question_dense_2 = tf.keras.layers.Dense(1000, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=28))(image_question_dense_1)
#     output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, kernel_initializer = tf.keras.initializers.glorot_normal(seed=15))(image_question_dense_2)
    
#     # out = Dense(1, activation='sigmoid')(clf_output)
    
#     model = Model(inputs=[input_word_ids, input_mask, segment_ids, image_input], outputs=output)
#     model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
#     return model

In [11]:
class AttentionMaps(tf.keras.layers.Layer):
  """
  Given an image feature map V ∈ R(d×N), and the question representation Q ∈ R(d×T), 
  calculates the affinity matrix C ∈ R(T×N): C = tanh((QT)(Wb)V) ; 
  where Wb ∈ R(d×d) contains the weights. (Refer eqt (3) section 3.3).
  Given this affinity matrix C ∈ R(T×N), predicts image and question attention maps 
  (Refer eqt (4) section 3.3).
  Arguments:
    dim_k     : Hidden attention dimention
    reg_value : Regularization value
  Inputs:
    image_feat,    V : shape (N,  d) or (49, dim_d)
    ques_feat,     Q : shape (T,  d) or (23, dim_d)
  Outputs:
    Image and Question attention maps viz:
    a) Hv = tanh(WvV + (WqQ)C) and
    b) Hq = tanh(WqQ + (WvV )CT)
  """
  def __init__(self, dim_k, reg_value, **kwargs):
    super(AttentionMaps, self).__init__(**kwargs)

    self.dim_k = dim_k
    self.reg_value = reg_value

    self.Wv = Dense(self.dim_k, activation=None,\
                        kernel_regularizer=tf.keras.regularizers.l2(self.reg_value),\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=2))
    self.Wq = Dense(self.dim_k, activation=None,\
                        kernel_regularizer=tf.keras.regularizers.l2(self.reg_value),\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=3))

  def call(self, image_feat, ques_feat):
    """
    The main logic of this layer.
    """  

    # Affinity Matrix C
    # (QT)(Wb)V 
    C = tf.matmul(ques_feat, tf.transpose(image_feat, perm=[0,2,1])) # [b, 23, 49]
    # tanh((QT)(Wb)V)
    C = tf.keras.activations.tanh(C) 

    # (Wv)V
    WvV = self.Wv(image_feat)                             # [b, 49, dim_k]
    # (Wq)Q
    WqQ = self.Wq(ques_feat)                              # [b, 23, dim_k]

    # ((Wq)Q)C
    WqQ_C = tf.matmul(tf.transpose(WqQ, perm=[0,2,1]), C) # [b, k, 49]
    WqQ_C = tf.transpose(WqQ_C, perm =[0,2,1])            # [b, 49, k]

    # ((Wv)V)CT                                           # [b, k, 23]
    WvV_C = tf.matmul(tf.transpose(WvV, perm=[0,2,1]), tf.transpose(C, perm=[0,2,1]))  
                        
    WvV_C = tf.transpose(WvV_C, perm =[0,2,1])            # [b, 23, k]

    #---------------image attention map------------------
    # We find "Hv = tanh((Wv)V + ((Wq)Q)C)" ; H_v shape [49, k]

    H_v = WvV + WqQ_C                                     # (Wv)V + ((Wq)Q)C
    H_v = tf.keras.activations.tanh(H_v)                  # tanh((Wv)V + ((Wq)Q)C) 

    #---------------question attention map---------------
    # We find "Hq = tanh((Wq)Q + ((Wv)V)CT)" ; H_q shape [23, k]

    H_q = WqQ + WvV_C                                     # (Wq)Q + ((Wv)V)CT
    H_q = tf.keras.activations.tanh(H_q)                  # tanh((Wq)Q + ((Wv)V)CT) 
        
    return [H_v, H_q]                                     # [b, 49, k], [b, 23, k]
  
  def get_config(self):
    """
    This method collects the input shape and other information about the layer.
    """
    config = {
        'dim_k': self.dim_k,
        'reg_value': self.reg_value
    }
    base_config = super(AttentionMaps, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [12]:
class ContextVector(tf.keras.layers.Layer):
  """
  Method to find context vector of the image and text features
  (Refer eqt (4) and (5) section 3.3).
  
  Arguments:
    reg_value : Regularization value
    
  Inputs:
    image_feat V: image features, (49, d)
    ques_feat  Q: question features, (23, d)
    H_v: image attention map, (49, k)
    H_q: question attention map, (23, k)
  Outputs:
    Returns d-dimenstional context vector for image and question features
  """
  def __init__(self, reg_value, **kwargs):
    super(ContextVector, self).__init__(**kwargs)

    self.reg_value = reg_value

    self.w_hv = Dense(1, activation='softmax',\
                        kernel_regularizer=tf.keras.regularizers.l2(self.reg_value),\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=4))
    self.w_hq = Dense(1, activation='softmax',\
                        kernel_regularizer=tf.keras.regularizers.l2(self.reg_value),\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=5)) 
    

  def call(self, image_feat, ques_feat, H_v, H_q):
    """
    The main logic of this layer.
    """  
    # attention probabilities of each image region vn; a_v = softmax(wT_hv * H_v)
    a_v = self.w_hv(H_v)                               # [b, 49, 1]

    # attention probabilities of each word qt ;        a_q = softmax(wT_hq * H_q)
    a_q = self.w_hq(H_q)                               # [b, 23, 1]

    # context vector for image
    v = a_v * image_feat                               # [b, 49, dim_d]
    v = tf.reduce_sum(v, 1)                            # [b, dim_d]

    # context vector for question
    q = a_q * ques_feat                                # [b, 23, dim_d]
    q = tf.reduce_sum(q, 1)                            # [b, dim_d]


    return [v, q]

  def get_config(self):
    """
    This method collects the input shape and other information about the layer.
    """
    config = {
        'reg_value': self.reg_value
    }
    base_config = super(ContextVector, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [13]:
class PhraseLevelFeatures(tf.keras.layers.Layer):
  """
  We compute the phrase features by applying 1-D convolution on the word embedding 
  vectors with filters of three window sizes: unigram, bigram and trigram.
  The word-level features Qw are appropriately 0-padded before feeding into bigram and 
  trigram convolutions to maintain the length of the sequence after convolution.
  Given the convolution result, we then apply max-pooling across different n-grams at each word
  location to obtain phrase-level features
  (Refer eqt (1) and (2) section 3.2).
  Arguments:
    dim_d: hidden dimension
  Inputs:
    word_feat Q : word level features of shape (23, dim_d)
  Outputs:
    Phrase level features of the question of shape (23, dim_d)
  """
  def __init__(self, dim_d, **kwargs):
    super(PhraseLevelFeatures, self).__init__(**kwargs)
    
    self.dim_d = dim_d
    
    self.conv_unigram = Conv1D(self.dim_d, kernel_size=1, strides=1,\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=6)) 
    self.conv_bigram =  Conv1D(self.dim_d, kernel_size=2, strides=1, padding='same',\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=7)) 
    self.conv_trigram = Conv1D(self.dim_d, kernel_size=3, strides=1, padding='same',\
                            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=8)) 


  def call(self, word_feat):
    """
    The main logic of this layer.
    Compute the n-gram phrase embeddings (n=1,2,3)
    """
    # phrase level unigram features
    x_uni = self.conv_unigram(word_feat)                    

    # phrase level bigram features
    x_bi  = self.conv_bigram(word_feat)                     

    # phrase level trigram features
    x_tri = self.conv_trigram(word_feat)                    

    # Concat
    x = tf.concat([tf.expand_dims(x_uni, -1),\
                    tf.expand_dims(x_bi, -1),\
                    tf.expand_dims(x_tri, -1)], -1)         

    # https://stackoverflow.com/a/36853403
    # Max-pool across n-gram features; over-all phrase level feature
    x = tf.reduce_max(x, -1)                                

    return x

  def get_config(self):
    """
    This method collects the input shape and other information about the layer.
    """
    config = {
        'dim_d': self.dim_d
    }
    base_config = super(PhraseLevelFeatures, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [14]:
def build_model(bert_layer, max_len, dim_d, dim_k, l_rate, d_rate, reg_value):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    print(np.shape(sequence_output))
    print(np.shape(clf_output))

    image_input = tf.keras.Input(shape=(100,1024), batch_size=None, name="image_input")
    # image feature; (Wb)V                                          # [b, 49, dim_d]
    image_feat = Dense(dim_d, activation=None, name='Image_Feat_Dense', kernel_regularizer=tf.keras.regularizers.l2(reg_value), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=1))(image_input)
    image_feat = Dropout(d_rate, seed=1)(image_feat)

    Hv_w, Hq_w = AttentionMaps(dim_k, reg_value, name='AttentionMaps_Word')(image_feat, sequence_output)
    v_w, q_w = ContextVector(reg_value, name='ContextVector_Word')(image_feat, sequence_output, Hv_w, Hq_w)
    feat_w = tf.add(v_w,q_w)
    h_w = Dense(dim_d, activation='tanh', name='h_w_Dense', kernel_regularizer=tf.keras.regularizers.l2(reg_value), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=13))(feat_w)


    # phrase level
    text_feat_p = PhraseLevelFeatures(dim_d, name='PhraseLevelFeatures')(sequence_output)

    Hv_p, Hq_p = AttentionMaps(dim_k, reg_value, name='AttentionMaps_Phrase')(image_feat, text_feat_p)
    v_p, q_p = ContextVector(reg_value, name='ContextVector_Phrase')(image_feat, text_feat_p, Hv_p, Hq_p)
    feat_p = concatenate([tf.add(v_p,q_p), h_w], -1) 
    h_p = Dense(dim_d, activation='tanh', name='h_p_Dense', kernel_regularizer=tf.keras.regularizers.l2(reg_value), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=14))(feat_p)

    # sentence level
    text_feat_s = LSTM(dim_d, return_sequences=True, input_shape=(None, max_len, dim_d),\
                        kernel_initializer=tf.keras.initializers.glorot_uniform(seed=16))(text_feat_p)

    Hv_s, Hq_s = AttentionMaps(dim_k, reg_value, name='AttentionMaps_Sent')(image_feat, text_feat_s)
    v_s, q_s = ContextVector(reg_value, name='ContextVector_Sent')(image_feat, text_feat_p, Hv_s, Hq_s)
    feat_s = concatenate([tf.add(v_s,q_s), h_p], -1) 
    h_s = Dense(2*dim_d, activation='tanh', name='h_s_Dense',\
                    kernel_regularizer=tf.keras.regularizers.l2(reg_value),\
                        kernel_initializer=tf.keras.initializers.glorot_uniform(seed=15))(feat_s)

    z   = Dense(2*dim_d, activation='tanh', name='z_Dense',\
                    kernel_regularizer=tf.keras.regularizers.l2(reg_value),\
                        kernel_initializer=tf.keras.initializers.glorot_uniform(seed=16))(h_s)
    z   = Dropout(d_rate, seed=16)(z)

    # result
    result = Dense(1, activation='sigmoid')(z)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids, image_input], outputs=result)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    

    return model


In [15]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=False)

Wall time: 15.6 s


In [16]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [17]:
train_input = bert_encode(FMS.textNdesc.values, tokenizer, max_len=100)
train_labels = FMS.label.values

In [18]:
model = build_model(bert_layer, max_len = 100, dim_d = 1024, dim_k = 256, l_rate = 1e-4, d_rate = 0.3, reg_value = 0.01)
model.summary()

(None, 100, 1024)
(None, 1024)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 100, 1024)]  0           []                               
                                                                                                  
 Image_Feat_Dense (Dense)       (None, 100, 1024)    1049600     ['image_input[0][0]']            
                                                                                                  
 input_word_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 100)]        0           []                               
                                                               

c:\Users\yasir\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
%%time
checkpoint = ModelCheckpoint('model5.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    [train_input, img_feature], train_labels,
    validation_split=0.3,
    epochs=100 ,
    callbacks=[checkpoint],
    batch_size=32,
    steps_per_epoch=12   
)

Epoch 1/100
12/12 [==============================] - 1034s 91s/step - loss: 100.2229 - accuracy: 0.6250 - auc: 0.4977 - val_loss: 99.9012 - val_accuracy: 0.5866 - val_auc: 0.5698
Epoch 2/100
12/12 [==============================] - 968s 87s/step - loss: 99.6304 - accuracy: 0.6484 - auc: 0.5280 - val_loss: 99.3635 - val_accuracy: 0.6055 - val_auc: 0.5552
Epoch 3/100
12/12 [==============================] - 956s 86s/step - loss: 99.1213 - accuracy: 0.5807 - auc: 0.4992 - val_loss: 98.8004 - val_accuracy: 0.6055 - val_auc: 0.5776
Epoch 4/100
12/12 [==============================] - 1006s 90s/step - loss: 98.5483 - accuracy: 0.6458 - auc: 0.4673 - val_loss: 98.2777 - val_accuracy: 0.6055 - val_auc: 0.5607
Epoch 5/100
12/12 [==============================] - 960s 86s/step - loss: 98.0486 - accuracy: 0.5625 - auc: 0.5044 - val_loss: 97.7012 - val_accuracy: 0.6047 - val_auc: 0.5945
Epoch 6/100
12/12 [==============================] - 958s 86s/step - loss: 97.4407 - accuracy: 0.6797 - auc: 0.5